In [71]:
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np

MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [110]:
def get_polarity(sentence):
    return [str(val) for val in list(model(tokenizer.encode(sentence, return_tensors='tf'))[0][0].numpy())]

batch_max_size=32
def get_polarity_batch(batch,batch_max_size=32):
    polarities=list(get_polarity_batch_yield([entry[0] for entry in batch],batch_max_size))
    dates = [entry[1] for entry in batch]
    results = [(p,d) for d,p in zip(dates,polarities)]
    return results

def get_polarity_batch_yield(batch,batch_max_size=4):
    batch_size = len(batch)
    min=0
    max=batch_max_size
    while(min<batch_size):
        if max>=batch_size:
            max=batch_size-1
        for result in list(model(
            tokenizer.batch_encode_plus(
                batch[min:max],
                padding=True,
                truncation=True,
                return_attention_mask=True,
                max_length=64,
                return_tensors='tf'
            ))[0]):
            yield result.numpy().tolist()
        min+=batch_max_size
        max+=batch_max_size
texts = [("Hello, how are you?",123) for i in range(12)]
abc=list(get_polarity_batch(texts))
abc

[([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123),
 ([-1.821001648902893, 0.8904606103897095, 0.6783680319786072], 123)]